In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

## IO markup

### Train

In [2]:
from modules.data import bert_data

In [3]:
train_df_path = "/home/eartemov/ae/work/factRuEval-2016/dev.csv"
valid_df_path = "/home/eartemov/ae/work/factRuEval-2016/test.csv"

In [4]:
data = bert_data.LearnData.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels2.txt",
    clear_cache=True
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [5]:
from modules.models.bert_models import BERTAttnCRF

In [6]:
model = BERTAttnCRF.create(len(data.train_ds.idx2label), crf_dropout=0.3)

In [7]:
from modules.train.train import NerLearner

In [8]:
num_epochs = 100

In [9]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTAttnCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [10]:
model.get_n_trainable_params()

890617

In [ ]:
learner.fit(epochs=num_epochs)

### Predict

In [30]:
from modules.data.bert_data import get_data_loader_for_predict

In [31]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [32]:
preds = learner.predict(dl)

In [33]:
from sklearn_crfsuite.metrics import flat_classification_report

In [34]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [35]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [36]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=["I_ORG", "I_PER", "I_LOC"], digits=4)

In [37]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.8019    0.7415    0.7705      3865
       I_PER     0.9374    0.9569    0.9470      2112
       I_LOC     0.9007    0.7752    0.8333      1557

   micro avg     0.8620    0.8089    0.8346      7534
   macro avg     0.8800    0.8245    0.8503      7534
weighted avg     0.8603    0.8089    0.8330      7534



In [38]:
from modules.analyze_utils.main_metrics import precision_recall_f1

In [39]:
results = precision_recall_f1(true_labels, pred_labels)

processed 56409 tokens with 7534 phrases; found: 7070 phrases; correct: 6094.

precision:  86.20%; recall:  80.89%; FB1:  83.46

	LOC: precision:  90.07%; recall:  77.52%; F1:  83.33 1340

	ORG: precision:  80.19%; recall:  74.15%; F1:  77.05 3574

	PER: precision:  93.74%; recall:  95.69%; F1:  94.70 2156


